In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
import functools
from my_load_data import load_dataset_fn

In [2]:
#Parameters of photo
img_size = 256
num_channels = 3
num_classes = 5
batch_size = 20
#Hyper parameters for network
learning_rate = 1e-4
params = {"learning_rate": learning_rate,
          "img_size": img_size,
          "num_channels":num_channels,
          "num_classes":num_classes,
          "batch_size":batch_size}

In [3]:
class Model:
    
    def __init__(self, data, labels, params, mode=None):
        self.data = data
        self.target = labels
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.mode = mode
        self.prediction
        self.optimize
        self.error
        

    def prediction(self):
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels])
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = input_layer, 
                                 filters = 16,
                                 kernel_size = 5,
                                 strides = 1,
                                 padding = "same",
                                 activation = tf.nn.relu)
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")


        pool_1_flat = tf.reshape(pool1,[-1,int(self.img_size * self.img_size * 16 / 4)])


        dense = tf.layers.dense(inputs = pool_1_flat,
                                units = 128,
                                activation = tf.nn.relu)

        dense2 = tf.layers.dense(inputs = dense,
                                 units = 64,
                                 activation = tf.nn.relu)

        logits = tf.layers.dense(inputs = dense2,
                                 units = self.num_classes,
                                 activation = tf.nn.relu)




        # Softmax output of the neural network.
        #[10,x] tesnor
        y_pred = tf.nn.softmax(logits=logits)
        y_pred_cls = tf.argmax(y_pred, axis=1)
        return y_pred
    
    
    

    def optimize(self):
        cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target, logits=self.prediction)
        optimizer = tf.train.AdamOptimizer(learning_rate = params["learning_rate"])
        return optimizer.minimize(cross_entropy)


    def error(self):
        mistakes = tf.not_equal(
            tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
        return tf.reduce_mean(tf.cast(mistakes, tf.float32))
    
    
    


In [4]:
def main(params, mode = None):
    batch_size = params["batch_size"]
    dataset, labels = load_dataset_fn(1)
    image = tf.placeholder(tf.float32, [None, params["img_size"], params["img_size"], params["num_channels"]])
    label = tf.placeholder(tf.float32, [None, params["num_classes"]])
    model = Model(image, label, params, mode = None)
    num_steps = 100
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        print('Initialized')
        for step in range(num_steps):
            offset = (step*batch_size) % (labels.shape[0] - batch_size)
            batch_data = dataset[offset :(offset+ batch_size),:,:,:]
            batch_labels = labels[offset : (offset + batch_size),:]
            feed_dict = {image: dataset, label: labels}
            err = session.run(model.error, feed_dict= feed_dict)
            if (step % 1 == 0):
                print('Minibatch loss at step %d: %f' % (step, err))
                #print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
              

In [ ]:
tf.reset_default_graph()
if __name__ == '__main__':
    main(params)